In [3]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 5.2 MB/s eta 0:00:00:00:01


In [2]:
import pandas as pd
from openpyxl import load_workbook
import json

def create_workout_step(exercise, cat, step_id, reps, weight):
    """Helper function to create a workout step dictionary."""
    return {
        "type": "ExecutableStepDTO",
        "stepId": step_id,
        "stepOrder": step_id,
        "stepType": {
            "stepTypeId": 3 if exercise != "Rest" else 5,
            "stepTypeKey": "interval" if exercise != "Rest" else "rest",
            "displayOrder": 3 if exercise != "Rest" else 5
        },
        "childStepId": None,
        "description": None,
        "endCondition": {
            "conditionTypeId": 10,
            "conditionTypeKey": "reps",
            "displayOrder": 10,
            "displayable": True
        } if exercise != "Rest" else {
            "conditionTypeId": 1,
            "conditionTypeKey": "lap.button",
            "displayOrder": 1,
            "displayable": True
        },
        "endConditionValue": reps if exercise != "Rest" else None,
        "preferredEndConditionUnit": None,
        "endConditionCompare": None,
        "targetType": {
            "workoutTargetTypeId": 1,
            "workoutTargetTypeKey": "no.target",
            "displayOrder": 1
        } if exercise != "Rest" else None,
        "targetValueOne": None,
        "targetValueTwo": None,
        "targetValueUnit": None,
        "zoneNumber": None,
        "secondaryTargetType": None,
        "secondaryTargetValueOne": None,
        "secondaryTargetValueTwo": None,
        "secondaryTargetValueUnit": None,
        "secondaryZoneNumber": None,
        "endConditionZone": None,
        "strokeType": {
            "strokeTypeId": 0,
            "strokeTypeKey": None,
            "displayOrder": 0
        },
        "equipmentType": {
            "equipmentTypeId": 0,
            "equipmentTypeKey": None,
            "displayOrder": 0
        },
        "category": cat if exercise != "Rest" else None,
        "exerciseName": exercise if exercise != "Rest" else None,
        "workoutProvider": None,
        "providerExerciseSourceId": None,
        "weightValue": weight if exercise != "Rest" else None,
        "weightUnit": {
            "unitId": 8,
            "unitKey": "kilogram",
            "factor": 1000.0
        } if exercise != "Rest" else None
    }

# Load the workbook
wb = load_workbook('exercises.xlsx')

# Access the specific sheet
ws = wb['Sheet1']

# Access the table
table = ws.tables['Table1']

# Get the cell range of the table
table_range = table.ref
# Parse the start and end columns and rows from the table range
start_col = table.ref.split(':')[0][0]
start_row = int(table.ref.split(':')[0][1:])
end_col = table.ref.split(':')[1][0]
end_row = int(table.ref.split(':')[1][1:])

# Generate a list of column names
cols = [chr(i) for i in range(ord(start_col), ord(end_col)+1)]

# Concat the cols as string
cols = ','.join(cols)

# Load the data from the table's range into a DataFrame
df = pd.read_excel('exercises.xlsx', sheet_name='Sheet1', skiprows=start_row-1, nrows=end_row-start_row+1, usecols=cols)

# Initialize the base JSON structure
workout_json = {
    "workoutId": 860381734,
    "ownerId": 117096033,
    "workoutName": "Strength Workout (2)",
    "description": None,
    "updatedDate": "2024-03-12T04:58:47.0",
    "createdDate": "2024-03-12T04:58:47.0",
    "sportType": {
        "sportTypeId": 5,
        "sportTypeKey": "strength_training",
        "displayOrder": 5
    },
    "subSportType": None,
    "trainingPlanId": None,
    "author": {
        "userProfilePk": 117096033,
        "displayName": "624b98bb-e914-4484-aa71-64379b20690e",
        "fullName": "Steve De Peijper",
        "profileImgNameLarge": None,
        "profileImgNameMedium": "f1c7ed68-9f28-4979-84ec-6fba0d986faf-prfr.png",
        "profileImgNameSmall": "f1c7ed68-9f28-4979-84ec-6fba0d986faf-prth.png",
        "userPro": False,
        "vivokidUser": False
    },
    "sharedWithUsers": None,
    "estimatedDurationInSecs": 0,
    "estimatedDistanceInMeters": 0.0,
    "workoutSegments": [{
        "segmentOrder": 1,
        "sportType": {
            "sportTypeId": 5,
            "sportTypeKey": "strength_training",
            "displayOrder": 5
        },
        "poolLengthUnit": None,
        "poolLength": None,
        "avgTrainingSpeed": None,
        "estimatedDurationInSecs": None,
        "estimatedDistanceInMeters": None,
        "estimatedDistanceUnit": None,
        "estimateType": None,
        "description": None,
        "workoutSteps": []
    }],
    "poolLength": None,
    "poolLengthUnit": None,
    "locale": None,
    "workoutProvider": None,
    "workoutSourceId": None,
    "uploadTimestamp": None,
    "atpPlanId": None,
    "consumer": None,
    "consumerName": None,
    "consumerImageURL": None,
    "consumerWebsiteURL": None,
    "workoutNameI18nKey": None,
    "descriptionI18nKey": None,
    "avgTrainingSpeed": 0.0,
    "estimateType": None,
    "estimatedDistanceUnit": {
        "unitId": None,
        "unitKey": None,
        "factor": None
    },
    "workoutThumbnailUrl": None,
    "isSessionTransitionEnabled": None,
    "shared": False
}

# Dynamically add workout steps based on the Excel file
step_id = 1
for index, row in df.iterrows():
    # For the number of sets, repeat the exercise and increment the weight by the increment
    for i in range(row['Sets']):
        workout_step = create_workout_step(row['Exercise'], row['cat'], step_id, row['Reps'], row['Weight'] + i * row['Increment'])    
        workout_json["workoutSegments"][0]["workoutSteps"].append(workout_step)
        step_id += 1
        # Add a rest step
        rest_step = create_workout_step("Rest", None, step_id, None, None)
        workout_json["workoutSegments"][0]["workoutSteps"].append(rest_step)
        step_id += 1

# Write the constructed workout JSON to a file
with open('workout.json', 'w') as file:
    json.dump(workout_json, file, indent=4)

print("Workout JSON file 'workout.json' has been successfully created.")


Workout JSON file 'workout.json' has been successfully created.
